## Hello world!

In [1]:
%load_ext Cython

In [2]:
%%cython 
print("Hello World!")

Hello World!


## Primes

http://docs.cython.org/src/tutorial/cython_tutorial.html#primes

In [9]:
def primes( kmax):
    p = [None]*1000
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [10]:
%%cython
def cprimes(int kmax):
    cdef int n, k, i
    cdef int p[1000]
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [11]:
primes(100) == cprimes(100)

True

In [13]:
n = 1000
%timeit primes(n)
%timeit cprimes(n)

1 loop, best of 3: 138 ms per loop
100 loops, best of 3: 3.57 ms per loop
